# Лабораторная работа 1

## Задание 1

### Описание

Реализовать клиентскую и серверную часть приложения. Клиент отсылает серверу
сообщение «Hello, server». Сообщение должно отразиться на стороне сервера.
Сервер в ответ отсылает клиенту сообщение «Hello, client». Сообщение должно
отобразиться у клиента.

- Обязательно использовать библиотеку `socket`
- Реализовать с помощью протокола UDP

Полезные ссылки:
-  https://habr.com/ru/post/149077/
-  https://andreymal.org/socket3/
-  https://docs.python.org/3.6/howto/sockets.html
-  https://docs.python.org/3.6/library/socket.html
-  https://www.youtube.com/watch?v=Lbfe3-v7yE0

### Код
#### Server

In [18]:
# %load "Task 1/server.py"
import socket


# UDP
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
# Ensures that port is always ready to be used again
sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
sock.bind(('localhost', 12346))

# Makes keyboard interrupt possible at all times
sock.settimeout(1.0)

print(
    f"Started server at udp://{sock.getsockname()[0]}:{sock.getsockname()[1]}")

while True:
    try:
        connection, client_address = None, None
        try:
            connection, client_address = sock.recvfrom(2048)
        # Handle timeout
        except IOError:
            continue

        data = connection.decode('utf-8')
        print('Recived:', data)

        sock.sendto(b"Hello, client", client_address)

    except KeyboardInterrupt:
        print("Stopping server...")
        if connection:
            connection.close()
        break
sock.close()


Started server at udp://127.0.0.1:12346
Recived: Hello, server
Recived: Hello, server
Recived: Hello, server
Stopping server...


#### Client
Так как Jupyter не хочет запускать cells в параллели, клиент запускается через командную строку:

python "./students/k33401/Reingeverts_Vadim/Lr1/Task 1/client.py"

In [5]:
# %load "Task 1/client.py"
import socket

# UDP
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.connect(('localhost', 12346))

sock.send(b"Hello, server")

try:
    connection = sock.recv(2048)
    data = connection.decode('utf-8')
    print('Recived:', data)
except ConnectionResetError:
    print("Could not connect to the server")

sock.close()


Could not connect to the server


## Задание 2

### Описание

Реализовать клиентскую и серверную часть приложения. Клиент запрашивает у
сервера выполнение математической операции, параметры, которые вводятся с
клавиатуры. Сервер обрабатывает полученные данные и возвращает результат
клиенту. Варианты:

1. **Теорема Пифагора**
2. Решение квадратного уравнения.
3. Поиск площади трапеции.
4. Поиск площади параллелограмма.

Вариант выбирается в соответствии с порядковым номером в журнале. Пятый
студент получает вариант 1 и т.д.

- Обязательно использовать библиотеку `socket`
- Реализовать с помощью протокола TCP

### Код
#### Server

In [19]:
# %load "Task 2/server.py"
import socket
from math import sqrt


def calc_pythagorean_equation(solveFor, x, y):
    solution = None
    x = float(x)
    y = float(y)
    
    if solveFor == "a":
        solution = sqrt(y**2 - x**2)
    elif solveFor == "b":
        solution = sqrt(y**2 - x**2)
    else:
        solution = sqrt(x**2 + y**2)
    return solution


# TCP
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# Ensures that port is always ready to be used again
sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
sock.bind(('localhost', 12346))


# Makes keyboard interrupt possible at all times
sock.settimeout(1.0)
sock.listen(10)

print(
    f"Started server at tcp://{sock.getsockname()[0]}:{sock.getsockname()[1]}")

while True:
    try:
        connection, client_address = None, None
        try:
            connection, client_address = sock.accept()
        # Handle timeout
        except IOError:
            continue
        data = connection.recv(2048)
        data = data.decode('utf-8')
        print('Recived:\n' + data)

        solveFor, x, y, _ = data.split("\n")
        solution = calc_pythagorean_equation(solveFor, x, y)

        print("Sending response:", solution)
        connection.send(str(solution).encode('utf-8'))

    except KeyboardInterrupt:
        print("Stopping server...")
        if connection:
            connection.close()
        break
sock.close()


Started server at tcp://127.0.0.1:12346
Stopping server...


#### Client
Так как Jupyter не хочет запускать cells в параллели, клиент запускается через командную строку:

python "./students/k33401/Reingeverts_Vadim/Lr1/Task 2/client.py"

In [20]:
# %load "Task 2/client.py"
import socket


def get_decimal_input(decimalName):
    decimalNum = None
    while True:
        try:
            decimalNum = float(input(f"Enter value for {decimalName}: "))
        except ValueError:
            print("Not a number.")
            continue
        if decimalNum < 0:
            print(f"{decimalName} must be a positive number.")
            continue
        else:
            break
    return decimalNum


# TCP
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
try:
    sock.connect(('localhost', 12346))

    print("\nPythagorean theorem solver.")
    print("""
        +
        |\\
        | \\ C
      A |  \\
        |   \\
        +----+
           B
    """)
    message = ""
    option1 = ""
    while True:
        option1 = input("Choose to solve for (A, B or C): ").lower()
        if option1 not in ('a', 'b', 'c'):
            print("Not an appropriate choice.")
        else:
            message += option1 + "\n"
            break
    if (option1 == "a"):
        b = get_decimal_input("B")
        c = get_decimal_input("C")

        message += str(b) + "\n"
        message += str(c) + "\n"

    elif (option1 == "b"):
        a = get_decimal_input("A")
        c = get_decimal_input("C")

        message += str(a) + "\n"
        message += str(c) + "\n"
    else:
        a = get_decimal_input("A")
        b = get_decimal_input("B")

        message += str(a) + "\n"
        message += str(b) + "\n"

    sock.send(message.encode("utf-8"))
    connection = sock.recv(2048)
    data = connection.decode('utf-8')

    print(f'\nSolution for {option1.upper()} is:', data)
except ConnectionRefusedError:
    print("Could not connect to the server")


sock.close()


Could not connect to the server


## Задание 3

### Описание

Реализовать серверную часть приложения. Клиент подключается к серверу. В ответ
клиент получает http-сообщение, содержащее html-страницу, которую сервер
подгружает из файла `index.html`.


Полезные ссылки:
-  http://zetcode.com/python/socket/
- Обязательно использовать библиотеку `socket`

### Код
#### Server

In [21]:
# %load "Task 3/server.py"
import socket
from os import path
from pathlib import Path
import webbrowser

# Makes consistent path to work directory in case of
# 1. Running .py file directly `python server.py`
# 2. Running .py file from another directory `python ./someComplicatedPath/server.py`
# 3. Running cell from .ipynb notebook

curr_dirname = None
ipynb_path = "./Task 3"
if "__file__" in globals():
    dirname = path.dirname(__file__)
else:
    dirname = path.abspath("") + ipynb_path

index_file = Path(dirname) / 'index.html'
# TCP
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# Ensures that port is always ready to be used again
sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
sock.bind(('localhost', 0))

# Makes keyboard interrupt possible at all times
sock.settimeout(1.0)
sock.listen(10)

url = f'http://{sock.getsockname()[0]}:{sock.getsockname()[1]}'
print(
    f"Started server at {url}")
webbrowser.open(url)

while True:
    try:
        connection, client_address = None, None
        try:
            connection, client_address = sock.accept()
        # Handle timeout
        except IOError:
            continue

        print("Incoming connection from:", client_address)

        response_type = "HTTP/1.1 200 OK\n"
        headers = "Content-Type: text/html; charset=utf-8\n\n"
        
        body = None
        with open(index_file, 'r', encoding="utf-8") as file:
            body = file.read()
            
        response = response_type + headers + body
        connection.sendall(response.encode('utf-8'))

    except KeyboardInterrupt:
        print("Stopping server...")
        if connection:
            connection.close()
        break
sock.close()


Started server at http://127.0.0.1:65188
Incoming connection from: ('127.0.0.1', 65189)
Stopping server...


## Задание 5

### Описание

Необходимо написать простой web-сервер для обработки GET и POST http
запросов средствами Python и библиотеки socket.

Базовый класс для простейшей реализации web-сервера доступен
[Google Doc](https://docs.google.com/document/d/1lv_3D9VtMxz8tNkA6rA1xu9zaWEIBGXiLWBo1cse-0k/edit?usp=sharing)

Подробный мануал по работе доступен
[iximiuz - Python Web Server](https://iximiuz.com/ru/posts/writing-python-web-server-part-3/)

Задание: сделать сервер, который может:
- Принять и записать информацию о дисциплине и оценке по дисциплине.
- Отдать информацию обо всех оценах по дисциплине в виде html-страницы.

### Код
#### Server

In [17]:
# %load "Task 5/server.py"
import socket
import sys


class MyHTTPServer:
    # Параметры сервера

    def __init__(self, host, port, name):
        self.host = host
        self.port = port
        self.name = name

    def serve_forever(self):
        # 1. Запуск сервера на сокете, обработка входящих соединений
        pass

    def serve_client(self):
        # 2. Обработка клиентского подключения
        pass

    def parse_request(self):
        # 3. функция для обработки заголовка http+запроса. Python, сокет
        # предоставляет возможность создать вокруг него некоторую обертку,
        # которая предоставляет file object интерфейс. Это дайте возможность
        # построчно обработать запрос. Заголовок всегда - первая строка.
        # Первую строку нужно разбить на 3 элемента  (метод + url + версия протокола).
        # URL необходимо разбить на адрес и параметры (isu.ifmo.ru/pls/apex/f?p=2143,
        # где isu.ifmo.ru/pls/apex/f, а p=2143 - параметр p со значением 2143)
        pass

    def parse_headers(self):
        # 4. Функция для обработки headers. Необходимо прочитать все заголовки после
        # первой строки до появления пустой строки и сохранить их в массив.
        pass

    def handle_request(self):
        # 5. Функция для обработки url в соответствии с нужным методом. В случае
        # данной работы, нужно будет создать набор условий, который обрабатывает GET
        # или POST запрос. GET запрос должен возвращать данные. POST запрос должен
        # записывать данные на основе переданных параметров.
        pass

    def send_response(self):
        # 6. Функция для отправки ответа. Необходимо записать в соединение status line
        # вида HTTP/1.1 <status_code> <reason>. Затем, построчно записать заголовки и
        # пустую строку, обозначающую конец секции заголовков.
        pass


if __name__ == '__main__':
    host = "localhost"
    port = "12346"
    name = "My Python Server"
    serv = MyHTTPServer(host, port, name)
    try:
        serv.serve_forever()
    except KeyboardInterrupt:
        pass


this is not main
